# We will detect pneumonia,a lung disease based on the x-rays we have. 

Chest X-rays dataset is taken from Kaggle which contain various x-rays images differentiated by two categories “Pneumonia” and “Normal”. I will be creating a deep learning model which will actually tell us whether the person is having pneumonia disease or not having pneumonia.

In [2]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob
from tensorflow.keras.optimizers.legacy import Adam

In [3]:
IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.


training_data = "C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\train"
testing_data ="C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\test" #Give our training and testing path


In [4]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [5]:
classes = glob("C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\train/*")

 As we deleted the first and the last columns in the previous step, We will just make a flattened layer and finally we just add our last layer with a softmax activation function. len(classes) indicate how many categories we have in our output layer.

In [6]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [7]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Compiling our model

In [8]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [11]:
training_set = train_datagen.flow_from_directory("C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\train",
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')


Found 5216 images belonging to 2 classes.


In [12]:
test_set = testing_datagen.flow_from_directory("C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\test",
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


# Creating our model with epoch 5

In [13]:
fitted_model = final_model.fit_generator( #Fitting the model.
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

C:\Users\MD AREEB  AKHTAR\AppData\Local\Temp\ipykernel_18756\1085249960.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator( #Fitting the model.


Epoch 1/5
1304/1304 [==============================] - 893s 683ms/step - loss: 0.2624 - accuracy: 0.9266 - val_loss: 0.5100 - val_accuracy: 0.9006
Epoch 2/5
1304/1304 [==============================] - 851s 652ms/step - loss: 0.2390 - accuracy: 0.9480 - val_loss: 1.3120 - val_accuracy: 0.8413
Epoch 3/5
1304/1304 [==============================] - 851s 652ms/step - loss: 0.1978 - accuracy: 0.9571 - val_loss: 0.9160 - val_accuracy: 0.8622
Epoch 4/5
1304/1304 [==============================] - 1004s 770ms/step - loss: 0.2276 - accuracy: 0.9519 - val_loss: 2.6130 - val_accuracy: 0.7692
Epoch 5/5
1304/1304 [==============================] - 1059s 812ms/step - loss: 0.2169 - accuracy: 0.9603 - val_loss: 1.2514 - val_accuracy: 0.8814


In [17]:
final_model.save('our_model.h5')


C:\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing our model against data which we have

In [18]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img("C:\\Users\\MD AREEB  AKHTAR\\OneDrive\\Desktop\\archive\\chest_xray\\val\\PNEUMONIA\\person1954_bacteria_4886.jpeg",target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 0s 399ms/step
Person is affected with Pneumonia.
Predictions: [[1.3112796e-23 1.0000000e+00]]
